# <center> 군집별로 알아보는 코로나의 영향 </center>

## __[목차]__

### 1. 사용데이터 정의
    1-1. 내부데이터
    1-2. 외부데이터
    1-3. 군집화데이터

### 2. 데이터 분석 및 시각화 방법

    2-1. 군집화 방법
    2-2. 변화추이 계산 방법
    2-3. 

### 3. 인사이트 분석

    3-1. 해석에 필요한 주요 이슈들
    3-2. 군집별 시각화 해석
    3-3. 인사이트 정리

### 4. 한계 및 추가 연구 내용

    4-1. 시각화의 한계점
    4-2. 추가 연구 내용

<img src="http://scimonitors.com/wp-content/uploads/2020/03/banner.png"  align = "center" > 

In [1]:
import os
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

# 1. 사용데이터 정의

In [2]:
# directory 정의
BASE_DIR = '../data/'
INTERNAL_DIR = 'internal_data'
EXTERNAL_DIR = 'external_data'
MERGE_DATA = 'merge_data'

In [3]:
category_dict = {'버스': '대중교통',
 '지하철': '대중교통',
 '택시': '대중교통',
 '국산신차': '교통',
 '렌트카': '교통',
 '전기차충전소': '교통',
 '주차장': '교통',
 '주유소': '교통',
 '자동차시트/타이어': '교통',
 '카인테리어': '교통',
 '중고자동차': '교통',
 '이륜차판매': '교통',
 '수입자동차': '교통',
 '캐쥬얼의류': '의류',
 '정장': '의류',
 '기타의류': '의류',
 '신발': '의류',
 '스포츠의류': '의류',
 '내의판매점': '의류',
 '맞춤복점': '의류',
 '와이셔츠/타이': '의류',
 '인터넷Mall': '온라인',
 '인터넷P/G': '온라인',
 '인터넷종합Mall': '온라인',
 '제과점': '음식점',
 '중국음식': '음식점',
 '서양읍식': '음식점',
 '스넥': '음식점',
 '일반한식': '음식점',
 '일식회집': '음식점',
 '한정식': '음식점',
 '기타건강식': '음식점',
 '당구장': '실내활동',
 '볼링장': '실내활동',
 '스크린골프': '실내활동',
 '수영장': '실내활동',
 '영화관': '실내활동',
 '카지노': '실내활동',
 '노래방': '실내활동',
 '수족관': '실내활동',
 '레져업소(회원제형태)': '실내활동',
 '헬스크럽': '실내활동',
 '여객선': '관광',
 '관광여행': '관광',
 '항공사': '관광',
 '2급 호텔': '관광',
 '1급호텔': '관광',
 '콘도': '관광',
 '기타숙박업': '관광',
 '종합레져타운': '관광',
 '한의원': '의료',
 '종합병원': '의료',
 '한약방': '의료',
 '의료용품': '의료',
 '치과의원': '의료',
 '치과병원': '의료',
 '병원': '의료',
 '한방병원': '의료',
 '약국': '의료',
 '기타의료기관 및 기타의료기기': '의료',
 '칵테일바': '유흥주점',
 '유흥주점': '유흥주점',
 '단란주점': '유흥주점',
 '테니스장': '실외활동',
 '골프연습장': '실외활동',
 '기타교육': '교육',
 '유치원': '교육',
 '기능학원': '교육',
 '보습학원': '교육',
 '학원(회원제형태)': '교육',
 '예체능학원': '교육',
 '외국어학원': '교육',
 '비료/농약/사료/종자': '농업',
 '기타농업관련': '농업',
 '농기계': '농업'}

In [4]:
def find_csv_filenames( path_to_dir, suffix=".csv" ):
    filenames = os.listdir(path_to_dir)
    return [filename for filename in filenames if filename.endswith(suffix)]


def make_datetime(x):
    x = str(x)
    year = x[:4]
    month = x[4:6]
    day = x[6:]
    date = f'{year}-{month}-{day}'
    return date

### 1-1. 내부데이터 (Dacon 제공 데이터)
1. card.csv : 카드 소비 내역<br>
2. COVID_19/Time.csv : 코로나 확진자 수<br>

#### 카드 소비 내역

In [5]:
COVID_DIR = 'COVID_19'
DATA_DIR = 'data'

CARD_FILENAME = 'card_20200717.csv'
COVID_FILENAME = 'Time.csv'
CATEGORISING_FILENAME = 'dacon_categorising.xlsx'

COVID_PATH = os.path.join(BASE_DIR, INTERNAL_DIR, COVID_DIR, COVID_FILENAME)

In [6]:
card_df = pd.read_csv(os.path.join(BASE_DIR, INTERNAL_DIR, CARD_FILENAME))
card_df['mrhst_induty_cl_nm'] = card_df['mrhst_induty_cl_nm'].str.replace(pat = ' ', repl = '')
card_df.shape

(3713952, 7)

In [7]:
object_df = card_df[card_df['mrhst_induty_cl_nm'].isin(category_dict)] 
object_df['대분류'] = object_df['mrhst_induty_cl_nm'].replace(category_dict)
object_df.shape

# 임의로 만든 대분류를 새 컬럼에 할당함

(1348866, 8)

In [8]:
object_df = object_df.groupby(['receipt_dttm', '대분류', 'mrhst_induty_cl_nm'], as_index=False)['selng_cascnt'].sum()
# 일별 분류 기준 매출발생건수의 합

object_df.columns = ['datetime', '대분류', '중분류', 'sales count']
object_df = object_df.reset_index(drop = True)
object_df['datetime'] = object_df['datetime'].map(make_datetime)
object_df.head()

,datetime,대분류,중분류,sales count
0,2020-01-04,관광,1급호텔,38
1,2020-01-04,관광,관광여행,1063
2,2020-01-04,관광,기타숙박업,3738
3,2020-01-04,관광,항공사,4800
4,2020-01-04,교육,기능학원,36


In [10]:
internal_df = object_df.rename(columns={'datetime':'사용일자', 'sales count':'중분류_카운트'}) # , 'covid_confirmed':'확진자수'

In [11]:
check_df = internal_df.groupby(['중분류']).count()
check_df[check_df['사용일자']<50]

,사용일자,대분류,중분류_카운트
중분류,,,
카지노,3,3,3


In [12]:
internal_df = internal_df[internal_df['중분류'] != "카지노"]
# 사용일자가 50일이 되지 않는 데이터 제거

In [13]:
# internal_df['확진자수'] = internal_df['확진자수'].fillna(0)

In [14]:
internal_df.head()

,사용일자,대분류,중분류,중분류_카운트
0,2020-01-04,관광,1급호텔,38
1,2020-01-04,관광,관광여행,1063
2,2020-01-04,관광,기타숙박업,3738
3,2020-01-04,관광,항공사,4800
4,2020-01-04,교육,기능학원,36


### 1-2. 외부 데이터 (서울 열린데이터 광장, SEOUL TOPIS)
1. bus/data/*.csv : 버스 이용 데이터<br>
2. subway/data/*.csv : 지하철 이용 데이터 <br>
3. traffic/data/*.xlsx : 교통량 이용 데이터 <br>

#### 버스

In [15]:
EXTERNAL_DIR = 'external_data'
EXTERNAL_PATH = os.path.join('..', DATA_DIR, EXTERNAL_DIR)

EXTERNAL_DIR = [os.path.join(EXTERNAL_PATH, name, DATA_DIR) for name in os.listdir(EXTERNAL_PATH) if os.path.isdir(os.path.join(EXTERNAL_PATH, name))]
EXTERNAL_DIR

['../data/external_data/subway/data',
 '../data/external_data/bus/data',
 '../data/external_data/traffic/data']

In [16]:
SUBWAY_PATH = EXTERNAL_DIR[0]
BUS_PATH = EXTERNAL_DIR[1]
TRAFFIC_DIR = EXTERNAL_DIR[2]

SUBWAY_PATH, BUS_PATH, TRAFFIC_DIR

('../data/external_data/subway/data',
 '../data/external_data/bus/data',
 '../data/external_data/traffic/data')

In [17]:
data_list = list()
bus_files = find_csv_filenames(BUS_PATH)

for csv_file in bus_files:
    data_list.append(os.path.join(BUS_PATH, csv_file))
    
data_list.sort()

In [18]:
bus_df = pd.DataFrame(columns=['사용일자','승차총승객수'])
bus_df_kor = pd.DataFrame(columns=['사용일자', '노선번호', '노선명', '버스정류장ARS번호', '역명', '승차총승객수', '하차총승객수', '등록일자'])
bus_df_eng = pd.DataFrame(columns=['use_dt', 'bus_route_id', 'bus_route_no', 'bus_route_nm', 'bsst_ars_no', 'bus_sta_nm', 'ride_pasgr_num', 'alight_pasgr_num', 'work_dt'])
# 외부 데이터가 5월부터 컬럼명이 영문으로 변경되어 데이터프레임 두 개를 생성

In [19]:
for file in data_list[:4]:
    df = pd.read_csv(file, encoding = 'cp949', index_col = False)
    bus_df_kor = bus_df_kor.append(df, ignore_index=True)
    
for file in data_list[4:]:
    df = pd.read_csv(file, encoding = 'cp949', index_col = False)
    bus_df_eng = bus_df_eng.append(df, ignore_index=True)

In [20]:
bus_df_kor = bus_df_kor[['사용일자','승차총승객수']]
bus_df_eng = bus_df_eng.rename(columns={'use_dt':'사용일자','ride_pasgr_num':'승차총승객수'})[['사용일자','승차총승객수']]

In [21]:
bus_df = pd.DataFrame(columns=['사용일자','승차총승객수'])

bus_df = bus_df.append(bus_df_kor)
bus_df = bus_df.append(bus_df_eng)

In [23]:
bus_df = bus_df.groupby(['사용일자']).sum().reset_index()
bus_df = bus_df.rename(columns={'승차총승객수':'중분류_카운트'})

bus_df['사용일자'] = bus_df['사용일자'].map(make_datetime)
bus_df['중분류'] = "버스"

In [25]:
bus_df.head()

,사용일자,중분류_카운트,중분류
0,2020-01-01,2509091,버스
1,2020-01-02,5286628,버스
2,2020-01-03,5580078,버스
3,2020-01-04,4036695,버스
4,2020-01-05,3138649,버스


#### 지하철

In [26]:
data_list = list()
subway_files = find_csv_filenames(SUBWAY_PATH)

for csv_file in subway_files:
    data_list.append(os.path.join(SUBWAY_PATH, csv_file))
    
data_list.sort()

In [27]:
subway_list = list()

for file in data_list:
    try:
        tmp_df = pd.read_csv(file, encoding = 'cp949', index_col = False)
    except:
        tmp_df = pd.read_csv(file, encoding = 'utf-8', index_col = False)

    subway_list.append(tmp_df)

In [28]:
subway_df = pd.concat(subway_list)
subway_df.shape

(107483, 7)

In [29]:
subway_df = subway_df.groupby('사용일자', as_index = False)['승차총승객수'].sum()
subway_df = subway_df.rename(columns={'승차총승객수':'중분류_카운트'})

subway_df['사용일자'] = subway_df['사용일자'].map(make_datetime)
subway_df['중분류_카운트'] = subway_df['중분류_카운트'].map(lambda x : int(x))
subway_df['중분류'] = '지하철'

In [30]:
subway_df.head()

,사용일자,중분류_카운트,중분류
0,2020-01-01,3479698,지하철
1,2020-01-02,7691529,지하철
2,2020-01-03,8224917,지하철
3,2020-01-04,6042529,지하철
4,2020-01-05,4425521,지하철


#### 교통량

In [31]:
data_list = []

filenames = find_csv_filenames(TRAFFIC_DIR, 'xlsx')
for csv_file in filenames:
    data_list.append(csv_file)

In [32]:
traffic_df = pd.DataFrame(columns=['일자', '요일', '지점명', '지점번호', '방향', '구분', '0시', '1시', '2시', '3시', '4시',
       '5시', '6시', '7시', '8시', '9시', '10시', '11시', '12시', '13시', '14시', '15시',
       '16시', '17시', '18시', '19시', '20시', '21시', '22시', '23시'])

In [33]:
month_list = ['{:0>2}'.format(num) for num in range(1, 7)]
month_list

['01', '02', '03', '04', '05', '06']

In [34]:
traffic_file_path = os.path.join(TRAFFIC_DIR, '{}월 서울시 교통량 조사자료.xlsx')
traffic_file_path

'../data/external_data/traffic/data/{}월 서울시 교통량 조사자료.xlsx'

In [35]:
for month in month_list:
    traffic_df = traffic_df.append(pd.read_excel(traffic_file_path.format(month), sheet_name=f'{2020}년 {month}월'))

In [37]:
traffic_df.head()

,일자,요일,지점명,지점번호,방향,구분,0시,1시,2시,3시,...,15시,16시,17시,18시,19시,20시,21시,22시,23시,요일(2)
0,20200101,수,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,712.0,645.0,437.0,309.0,...,1416.0,1483.0,1329.0,1157.0,1014.0,954.0,849.0,780.0,480.0,NaN
1,20200102,목,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,315.0,222.0,186.0,165.0,...,1897.0,1842.0,2061.0,1994.0,1443.0,1233.0,1165.0,1094.0,852.0,NaN
2,20200103,금,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,632.0,457.0,295.0,236.0,...,1929.0,2049.0,2140.0,2178.0,1654.0,1356.0,1260.0,1253.0,941.0,NaN
3,20200104,토,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,740.0,518.0,388.0,331.0,...,1788.0,1588.0,1669.0,1530.0,1222.0,1143.0,1089.0,1039.0,791.0,NaN
4,20200105,일,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,533.0,424.0,297.0,230.0,...,1673.0,1494.0,1429.0,1288.0,1035.0,987.0,884.0,803.0,564.0,NaN


In [38]:
traffic_df = traffic_df.groupby('일자')[[name for name in traffic_df.columns if name[0].isnumeric()]].sum()
traffic_df.head()

,0시,1시,2시,3시,4시,5시,6시,7시,8시,9시,...,14시,15시,16시,17시,18시,19시,20시,21시,22시,23시
일자,,,,,,,,,,,,,,,,,,,,,
20200101,266533.0,296321.0,185636.0,134134.0,114799.0,125439.0,153618.0,177253.0,229955.0,271736.0,...,448721.0,463652.0,465493.0,440349.0,397753.0,372566.0,369655.0,343421.0,271340.0,185353.0
20200102,128132.0,85712.0,64405.0,61816.0,102125.0,228184.0,417027.0,543584.0,576058.0,531913.0,...,513649.0,525513.0,537179.0,553531.0,550529.0,482317.0,416175.0,405076.0,359995.0,275395.0
20200103,205548.0,153871.0,114252.0,96035.0,119189.0,223213.0,398420.0,538560.0,585920.0,548984.0,...,538689.0,544406.0,554737.0,564720.0,554247.0,515284.0,464737.0,440925.0,407617.0,328408.0
20200104,257111.0,203631.0,164033.0,133204.0,137314.0,201180.0,281633.0,320162.0,417084.0,489365.0,...,530548.0,531122.0,535347.0,539571.0,496455.0,424779.0,401285.0,400037.0,353366.0,274282.0
20200105,215442.0,164669.0,126931.0,100601.0,94797.0,118228.0,166274.0,205482.0,295470.0,396652.0,...,511018.0,511254.0,509879.0,487306.0,442948.0,393968.0,385019.0,357217.0,290775.0,206582.0


In [39]:
traffic_df = traffic_df.T.sum().reset_index()
traffic_df = traffic_df.rename(columns={'일자':'사용일자', 0:'중분류_카운트'})

traffic_df['사용일자'] = traffic_df['사용일자'].map(make_datetime)
traffic_df['중분류_카운트'] = traffic_df['중분류_카운트'].map(lambda x : int(x))
traffic_df['중분류'] = '교통량'

In [41]:
traffic_df.head()

,사용일자,중분류_카운트,중분류
0,2020-01-01,7347299,교통량
1,2020-01-02,9326734,교통량
2,2020-01-03,9998638,교통량
3,2020-01-04,9198233,교통량
4,2020-01-05,7868358,교통량


========================================

In [42]:
external_df = bus_df.append(subway_df)
external_df = external_df.append(traffic_df)
external_df.head()

,사용일자,중분류_카운트,중분류
0,2020-01-01,2509091,버스
1,2020-01-02,5286628,버스
2,2020-01-03,5580078,버스
3,2020-01-04,4036695,버스
4,2020-01-05,3138649,버스


### 1-3. 최종 데이터 (내부데이터 + 외부데이터)

- Internal + External + Covid Confirmation Count = Result<br>

In [43]:
external_df['대분류'] = np.nan

In [44]:
external_df.loc[external_df['중분류'] == '교통량', ['대분류']] = '교통'
external_df.loc[external_df['중분류'] != '교통량', ['대분류']] = '대중교통'

In [45]:
concated_df = pd.concat([internal_df, external_df], axis = 0)
concated_df

,사용일자,대분류,중분류,중분류_카운트
0,2020-01-04,관광,1급호텔,38
1,2020-01-04,관광,관광여행,1063
2,2020-01-04,관광,기타숙박업,3738
3,2020-01-04,관광,항공사,4800
4,2020-01-04,교육,기능학원,36
...,...,...,...,...
177,2020-06-26,교통,교통량,10115813
178,2020-06-27,교통,교통량,9465922
179,2020-06-28,교통,교통량,8023217
180,2020-06-29,교통,교통량,9589932


In [46]:
median_v_series = concated_df.groupby(['대분류', '중분류'], as_index = False)['중분류_카운트'].median()
# 분류 별 카운트의 중앙값 할당
median_v_series.rename(columns = {'중분류_카운트' : '중분류_대표값'}, inplace = True)

In [47]:
combined_median_df = pd.merge(concated_df, median_v_series, on = ['대분류', '중분류'])
combined_median_df.head(2)

,사용일자,대분류,중분류,중분류_카운트,중분류_대표값
0,2020-01-04,관광,1급호텔,38,338.0
1,2020-01-06,관광,1급호텔,19,338.0


In [48]:
combined_median_df['증가추이'] = ((combined_median_df['중분류_카운트']-combined_median_df['중분류_대표값']) / combined_median_df['중분류_대표값']) * 100

In [56]:
combined_median_df[(combined_median_df['중분류_대표값'] == 0) & (combined_median_df['중분류_카운트'] == 0)]

,사용일자,대분류,중분류,중분류_카운트,중분류_대표값,증가추이


#### 코로나 

In [ ]:
covid_df = pd.read_csv(COVID_PATH)
covid_df.head() # (163, 7)

In [51]:
covid_confirmation_df = covid_df[['date', 'confirmed']]
covid_confirmation_df.columns = ['사용일자', '확진자수']
# 확진자 수만 차출

covid_confirmation_df.head()

,사용일자,확진자수
0,2020-01-20,1
1,2020-01-21,1
2,2020-01-22,1
3,2020-01-23,1
4,2020-01-24,2


In [54]:
result_df = combined_median_df.merge(covid_confirmation_df, on='사용일자', how='left')
result_df['확진자수'] = result_df['확진자수'].fillna(0)
result_df.head()

,사용일자,대분류,중분류,중분류_카운트,중분류_대표값,증가추이,확진자수
0,2020-01-04,관광,1급호텔,38,338.0,-88.757396,0.0
1,2020-01-06,관광,1급호텔,19,338.0,-94.378698,0.0
2,2020-01-07,관광,1급호텔,40,338.0,-88.165680,0.0
3,2020-01-10,관광,1급호텔,43,338.0,-87.278107,0.0
4,2020-01-11,관광,1급호텔,59,338.0,-82.544379,0.0
